In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import sqlite3
from datetime import date
import hashlib
import sys
import os
import re
import requests as rq

project_dir = '/home/mi/Dropbox/prog/python/tradeDB'
os.chdir(project_dir)

from importlib import reload
from workers import sql
from workers import api_stooq
from workers.common import read_json
from workers.common import get_cookie

In [2]:
SECTORS = {
            "INDEXES": {"file": "./assets/indexes.json"},
            "STOCK": {"file": "./assets/stock.json"},
            "ETF": {"file": "./assets/etf.json"},
            "COMODITIES": {"file": "./assets/comodities.json"},
        }
for k in SECTORS:
    SECTORS[k]["data"] = read_json(
        SECTORS[k]["file"]
    )

In [16]:
reload(api_stooq)
#region = "EUROPE & CENTRAL ASIA"
#region = "U.S."
#region = "CANADA"
#region = "U.S. Main"
#region="LATIN AMERICA & CARIBBEAN"
region = "MIDDLE EAST & NORTH AFRICA"
api = SECTORS["INDEXES"]["data"][region]["api"]  # type: ignore
dat = api_stooq.web_stooq(
    sector_id=api["id"],  # type: ignore
    sector_grp=api["group"],  # type: ignore
    symbol="%",
    from_date=date.today(),
    end_date=date.today(),
)
dat.to_csv("./dev/tab6.csv")

In [18]:
dat=pd.read_csv("./dev/tab6.csv")
tab = "INDEXES"
db = './trader.sqlite'
names=dat['name']

In [17]:
dat

,symbol,name,val,date,unnamed: 6,unnamed: 7,unnamed: 8
63,^TASI,TADAWUL ALL SHARE INDEX - SAUDI ARABIA,10411.26,2023-03-05,NaN,NaN,NaN


In [19]:
countries = sql.get_sql(
            tab="GEO", get="country", search=["%"], db_file=db
        )["country"]["country"]

In [20]:
names = [re.sub(r"WIG.*$", n+r" - POLAND", n) for n in names]
names = [re.sub(r"ATX.*$", r"ATX - AUSTRIA", n) for n in names]

In [21]:
split = [re.split(" - ", n) for n in names]
name_short = [s[0] for s in split]
# just small cleaning
name_short = [re.sub(r"INDEX", "", n).strip() for n in name_short]
name_country = []
for s in split:
    if len(s) > 1:
        name_country.append(s[1])  
    else:
        name_country.append("null")  


In [22]:
# simplify countries - special cases
name_country = [re.sub(r"SOUTH KOREA", r"KOREA, REP.", c) for c in name_country]
name_country = [re.sub(r"SLOVAKIA", r"SLOVAK REPUBLIC", c) for c in name_country]
name_country = [re.sub(r"SWISS", r"SWITZERLAND", c) for c in name_country]
name_country = [re.sub(r"TURKEY", r"TURKIYE", c) for c in name_country]
name_country = [re.sub(r"U.S.", r"UNITED STATES", c) for c in name_country]

In [23]:
match = [re.search(c, r"-".join(countries)) for c in name_country]
        # handle what not found
for i in range(len(match)):
    if not match[i]:
        name_short[i] = names[i]
        name_country[i] = "null"

In [24]:
# search of iso codes needs to be in loop
        # otherway will be unique in alphabetical order and missing empty rows
resp = [
    sql.get_sql(
        tab="GEO",
        get="iso2",
        search=[n + "%"],
        cols=["country"],
        db_file=db,
    )
    for n in name_country
]

In [25]:
iso2 = []
for r in resp:
    i = r['country'].iloc[0,0]
    if not i:
        iso2.append("")
    else:
        iso2.append(i)


In [26]:
pd.DataFrame(name_short,iso2)

,0
SA,TADAWUL ALL SHARE


In [101]:
data=pd.read_csv("./dev/tab6.csv")
grp=api['group']

In [107]:
grp='BRAZIL'

In [108]:
grp = grp.split(';')

In [109]:
for i,g in enumerate(grp):
    if g[0]=='-':
        grp[i] = "^(?!.*"+g[1:]+"$).*$"
    else:
        grp[i] = '.*'+g+'$'


In [110]:
grp

['.*BRAZIL$']

In [111]:
rows = [data['name'].apply(lambda x: re.search(g,x) is not None) for g in grp]
rows = np.logical_and.reduce(rows)

In [112]:
data.loc[rows,:]

,Unnamed: 0,symbol,name,val,date
0,0,^BVP,BOVESPA INDEX - BRAZIL,103865.99,2023-03-03


In [143]:
grpNameRows = data.iloc[:, 1] == data.iloc[:, 2]
grpName = data.loc[grpNameRows, "name"].to_frame()
if grpName.empty:
    # we dont have groups so we have suffixes
    grp = grp.split(';')
    for i,g in enumerate(grp):
        if g[0]=='-':
            grp[i] = "^(?!.*"+g[1:]+"$).*$" # i.e. '^(?!.*CANADA$).*$'
        else:
            grp[i] = '.*'+g+'$' # i.e. '.*CANADA$'
    grp_rows = [data['name'].apply(lambda x: re.search(g,x) is not None) for g in grp]
    grp_rows = np.logical_and.reduce(grp_rows)
    data = data.loc[grp_rows,:]
else:
    start = list(map(lambda x: x + 1, grpName.index.to_list()))
    end = list(map(lambda x: x - 1, grpName.index.to_list()))  # shift by one
    end.append(len(data))

    grpName["Start"] = start
    grpName["End"] = end[1:]
    grpRow = grpName.loc[:, "Name"] == grp
    data = data.loc[
    grpName["Start"].values[grpRow][0] : grpName["End"].values[grpRow][0], :
    ]


In [132]:
STOOQ_COOKIE = "./assets/header_stooq.json"
cookie = get_cookie(STOOQ_COOKIE)
reload(api_stooq)

<module 'workers.api_stooq' from '/home/mi/Dropbox/prog/python/tradeDB/workers/api_stooq.py'>

In [146]:
components='wig20'
url = f"https://stooq.com/q/i/?s={components}&l=%page%&i"

In [140]:
api = SECTORS['INDEXES']['data']['EUROPE & CENTRAL ASIA']['api']
sector_id = api['id']
url = f"https://stooq.com/t/?i={sector_id}&v=0&l=%page%&f=0&n=1&u=1"

In [147]:
url = re.sub("%page%", str(i), url)
resp = rq.get(url, headers=cookie)

In [152]:
page = bs(resp.content, "lxml")
htmlTab = page.find(id="fth1")
pdTab = pd.read_html(htmlTab.prettify())[0]  # type: ignore

In [153]:
if pdTab.columns.nlevels >1:
    pdTab = pdTab.droplevel(0, axis='columns')

In [154]:

# some basic formatting: str_to_lower
pdTab.rename(str.lower, axis="columns", inplace=True)
# rename polish to english
pdTab.rename(columns={"nazwa": "name","kurs":'val','data':'date', 'zmiana':'change'}, inplace=True)
# remove 'change' col, rename other columns (Last->val), 
pdTab = pdTab.loc[:, [re.search("(change|kapitalizacja|warto|zysk|ttm|c/wk|stopa)",c) is None for c in pdTab.columns]]
pdTab.rename(columns={"last": "val"}, inplace=True)
# convert dates
pdTab["date"] = api_stooq.convert_date(pdTab["date"])


In [163]:
pdTab.loc[:,['symbol','name']].to_records(index=False)

rec.array([('ACP', 'ASSECOPOL'), ('ALE', 'ALLEGRO'), ('CCC', 'CCC'),
           ('CDR', 'CDPROJEKT'), ('CPS', 'CYFRPLSAT'), ('DNP', 'DINOPL'),
           ('JSW', 'JSW'), ('KGH', 'KGHM'), ('KRU', 'KRUK'),
           ('KTY', 'KETY'), ('LPP', 'LPP'), ('MBK', 'MBANK'),
           ('OPL', 'ORANGEPL'), ('PCO', 'PEPCO'), ('PEO', 'PEKAO'),
           ('PGE', 'PGE'), ('PKN', 'PKNORLEN'), ('PKO', 'PKOBP'),
           ('PZU', 'PZU'), ('SPL', 'SANPL')],
          dtype=[('symbol', 'O'), ('name', 'O')])

In [165]:
for s,n in pdTab.loc[:,['symbol','name']].to_records(index=False):
    print(s,n)

ACP ASSECOPOL
ALE ALLEGRO
CCC CCC
CDR CDPROJEKT
CPS CYFRPLSAT
DNP DINOPL
JSW JSW
KGH KGHM
KRU KRUK
KTY KETY
LPP LPP
MBK MBANK
OPL ORANGEPL
PCO PEPCO
PEO PEKAO
PGE PGE
PKN PKNORLEN
PKO PKOBP
PZU PZU
SPL SANPL


In [ ]:
 d1 = pd.to_datetime(dates, errors="coerce")
year = date.today().strftime("%Y")
# to handle 'mmm d' we need add current year
d2 = pd.to_datetime(year + " " + dates, format="%Y %b %d", errors="coerce")
return d1.fillna(d2).dt.date
    

In [116]:
pdTab

,Symbol,Nazwa,Kurs,Zmiana,Data,Kapitalizacja (mln),Wartość księgowa (mln),Zysk na akcję (ttm),C/Z (ttm),C/WK,Stopa dyw. (%)
0,ACP,ASSECOPOL,79.600,+0.82%,3 mar,6606.82,6608.40,5.8102,13.7,1.00,4.2
1,ALE,ALLEGRO,28.740,+3.29%,3 mar,30375.45,8866.55,NaN,NaN,3.43,NaN
2,CCC,CCC,35.450,+2.22%,3 mar,1945.07,684.00,NaN,NaN,2.84,NaN
3,CDR,CDPROJEKT,130.660,-0.11%,3 mar,13166.71,1998.52,2.9763,43.9,6.59,0.8
4,CPS,CYFRPLSAT,17.350,+0.12%,3 mar,11096.12,15128.30,1.6845,10.3,0.73,6.0
5,DNP,DINOPL,382.700,+1.73%,3 mar,37519.91,3874.07,10.6011,36.1,9.68,NaN
6,JSW,JSW,53.000,+1.45%,3 mar,6222.81,14151.00,66.2500,0.8,0.44,NaN
7,KGH,KGHM,130.900,+2.23%,3 mar,26180.00,32206.00,33.5641,3.9,0.81,2.3
8,KRU,KRUK,327.600,+0.43%,3 mar,6328.84,3131.99,41.4684,7.9,2.02,3.9
9,KTY,KETY,505.000,+3.06%,3 mar,4873.33,1844.53,74.2647,6.8,2.64,10.4


In [17]:
page = bs(resp.content, "lxml")
htmlTab = page.find(id="fth1")
pdTab = pd.read_html(htmlTab.prettify())[0] # type: ignore


In [24]:
pdTab = pdTab.loc[:, ~pdTab.columns.str.startswith("Change")]
pdTab.rename(columns={"Last": "val"}, inplace=True)
pdTab.rename(str.lower, axis="columns", inplace=True)
pdTab["date"] = api_stooq.convert_date(pdTab["date"])
# pdTab.dropna(subset=["date"], inplace=True)  # in case 

In [25]:
pdTab

,symbol,name,val,date,unnamed: 6,unnamed: 7,unnamed: 8
0,Main,Main,Main,NaT,Main,Main,Main
1,ES.C,S&P 500 CASH,4045.64,2023-03-03,NaN,NaN,NaN
2,EW.C,S&P MIDCAP CASH,2648.27,2023-03-03,NaN,NaN,NaN
3,OP.C,S&P 1000 CASH,15045.72,2023-03-03,NaN,NaN,NaN
4,PC.C,S&P SMALLCAP CASH,1269.75,2023-03-03,NaN,NaN,NaN
5,Sector,Sector,Sector,NaT,Sector,Sector,Sector
6,BI.C,S&P 500 CONSUMER DISCRETIONARY CASH,1475.15,2023-03-03,NaN,NaN,NaN
7,BK.C,S&P 500 REAL ESTATE CASH,190.32,2023-03-03,NaN,NaN,NaN
8,BL.C,S&P 500 CONSUMER STAPLES CASH,729.93,2023-03-03,NaN,NaN,NaN
9,BM.C,S&P 500 ENERGY CASH,912.08,2023-03-03,NaN,NaN,NaN


In [27]:
url = f"https://stooq.com/t/?i=500&v=0&l=%page%&f=0&n=1&u=1"

In [29]:
re.sub("%page%",str(4),url)

'https://stooq.com/t/?i=500&v=0&l=4&f=0&n=1&u=1'